# Mackey Glass

In [1]:
def runForecastNew(connectivity, multi = 1, shift = 0, 
                training_ratio = 0.3, past_steps = 50,
                cheat_period = 100, cheat_steps = 60,
                save_fig = False,
                disable_tqdm = False,inList = 0, outList=0):

    N = connectivity.numOfWires
    inList = inList#np.argsort(connectivity.xa)[0:20]
    outList = outList#np.argsort(connectivity.xb)[::-1][0:20]
    
    theMat = np.zeros((N+2, N+2))
    theMat[0:N, 0:N] = connectivity.adj_matrix
    theMat[N, inList] = 1
    theMat[inList, N] = 1
    theMat[N+1, outList] = 1
    theMat[outList, N+1] = 1
    theGraph = nx.from_numpy_array(theMat)
    
    Connectivity = connectivity__(graph = theGraph)
    # junctionList = [findJunctionIndex(Connectivity, N+1, i) for i in outList]
    
    SimulationOptions = simulationOptions__(dt = 1e-2, T = 100,
                                        contactMode = 'preSet',
                                        electrodes = [N, N+1])
    signal = mkg_generator(10000, tau = 18, a = 0.2, b = 0.1, dt = 0.1)*multi+shift
    SimulationOptions.stimulus[0] = stimulus__(biasType = 'Custom', TimeVector = SimulationOptions.TimeVector, customSignal = signal)
    JunctionState = junctionState__(Connectivity.numOfJunctions, mode = 'binary', collapse = True)

    sim1, weight1, measure1 = forecast(SimulationOptions, Connectivity, JunctionState,
                                   training_ratio, past_steps, forecast_on = True, measure_type = 'voltage', 
                                   past_signal = True, pre_activate = False, 
                                   cheat_on = True, cheat_period = cheat_period, cheat_steps = cheat_steps,
                                   update_weight = False, update_stepsize = 100, disable_tqdm = disable_tqdm)
    
    RNMSE = getRNMSE(sim1.forecast[sim1.predict_index]-shift, sim1.stimulus[0].signal[sim1.predict_index]-shift)
    sim1.forecastParams = dict(numOfWires = sim1.numOfWires,
                                numOfJunctions = sim1.numOfJunctions,
                                multi = multi, shift = shift,
                                training_ratio = training_ratio,
                                past_steps = past_steps,
                                cheat_period = cheat_period,
                                forecast_steps = cheat_period - cheat_steps,
                                cheat_steps = cheat_steps)
    if save_fig and (RNMSE < 0.5):
        plt.figure()
        plotForecastPanel(sim1)
        import time
        filename = time.strftime("%Y-%m-%d-%H%M%S") + '_RNMSE_' + str(np.round(RNMSE, 4)) + '.png'
        plt.savefig(filename)
    
    logging.info(f'RNMSE = {RNMSE}')
    return RNMSE


In [ ]:
accuracy={'ASN300':[None]*len(ASN300),'Elegans':[],'WS Grid 300':[None]*len(ASN300),'WS Random 300':[None]*len(ASN300)}

In [ ]:
ASN_con_300=[None]*len(ASN300)
for i in range(len(ASN300)):
    ASN_con_300[i]=connectivity__(wires_dict=ASN300[i])   
# ASN_con_100=connectivity__(wires_dict=ASN100)   

In [ ]:
cheatSteps=range(10,61,10)
mse300_cheat=[[None]*len(ASN_con_300) for i in range(len(cheatSteps))]

for i in range(len(ASN_con_300)): #number of networks
    for j in range(len(cheatSteps)): #number of cheat steps
        print('Network ' + str(i))
        print('Cheat Steps ' + str(cheatSteps[j]))        
        #run forcast on network j for i cheat steps 
        mse300_cheat[j][i]=runForecast(ASN_con_300[i], multi = 1, shift = 0, 
                        training_ratio = 0.3, past_steps = 50,
                        cheat_period = 100, cheat_steps = cheatSteps[j],
                        save_fig = False,
                        disable_tqdm = False)

In [ ]:
accuracy['ASN300']=[[None]*len(ASN300) for i in range(len(cheatSteps))]
for i in range(len(cheatSteps)):
    for j in range(len(ASN300)):
        if i==0:
            ASN300[j]['Accuracy']['Mackey Glass']={'Cheat Steps':[],'Accuracy Value':[]}
        print(str(cheatSteps[i]))
        ASN300[j]['Accuracy']['Mackey Glass']['Cheat Steps'].append(cheatSteps[i])
        accuracy['ASN300'][i][j]=1-mse300_cheat[i][j]
        ASN300[j]['Accuracy']['Mackey Glass']['Accuracy Value'].append(1-mse300_cheat[i][j])
# for i in range(len(cheatSteps)):
#     for j in range(len(mse300_cheat[i])):
#         accuracy['ASN300'][i][j]=1-mse300_cheat[i][j] #save for analysis

#         ASN300[j]['Accuracy']['Mackey Glass']=accuracy['ASN300'][i][j] #save for plotting

In [ ]:
#Find best performing network:
best_params_nw=np.argsort(mse300_cheat) #sort by lowest MSE
best_accuracies=[None]*len(mse300_cheat)
for i in range(len(mse300_cheat)):
    best_accuracies[i]=np.array(mse300_cheat[i])[np.array(best_params_nw[i])] #define best accuracies based on lowest MSE

In [ ]:
#Plot accuracy vs number of cheat steps
meanAcc=[]
stdAcc=[]
for i in range(len(cheatSteps)):
    meanAcc.append(np.mean(accuracy['ASN300'][i]))
    stdAcc.append(np.std(accuracy['ASN300'][i]))
# plt.plot(meanAcc,'-o')
plt.figure(figsize=(10,10))
plt.errorbar(range(len(cheatSteps)),meanAcc,stdAcc, marker='^')
plt.xticks(range(0,len(ASN300[0]['Accuracy']['Mackey Glass']['Cheat Steps']),1),labels=ASN300[0]['Accuracy']['Mackey Glass']['Cheat Steps'])
plt.xlabel('Number of Cheat Steps')
plt.ylabel('Average ASN300 Accuracy')
plt.title('ASN 300 | Past Steps = 50 | Cheat Period = 100')
# plt.yscale('log')
plt.ylim([0, 1])

In [ ]:
#Current and Communicability Matrices:
commuMatASN_MG=[[None]*len(mse300_cheat) for i in range(len(mse300_cheat[0]))]
currMatASN_MG=[None]*len(mse300_cheat) for i in range(len(mse300_cheat[0]))]
for i in tqdm(range(len(mse300_cheat))): # for each cheat step
    for j in range(len(mse300_cheat[i])): # for each network
    commuMatASN_MG[j][i],currMatASN_MG[j][i]=commCurr(mse300_cheat[i][j])

## C. Elegans

In [ ]:
mseElegans=[]
cheat_steps=[]
#loop through cheat steps + see how accuracy changes
for i in cheatSteps:
    print('Cheat steps = ' + str(i))
    mseElegans.append(runForecastNew(connectivity__(graph=elegansGraph), multi = 1, shift = 0, 
                    training_ratio = 0.3, past_steps = 50,
                    cheat_period = 100, cheat_steps = i,
                    save_fig = False,
                    disable_tqdm = False, inList=8, outList=245))
    cheat_steps.append(i)
#change tau parameter for chaos

In [ ]:
Elegans['Accuracy']['Mackey Glass']['Cheat Steps']=[]
Elegans['Accuracy']['Mackey Glass']['Accuracy Value']=[]
accuracy['Elegans']=[]
Elegans_MG_accuracy=[[None]*len(Elegans) for i in range(len(cheatSteps))]

for i in range(len(mseElegans)):
    accuracy['Elegans']=1-mseElegans[i]
    Elegans['Accuracy']['Mackey Glass']['Cheat Steps'].append(cheat_steps[i])
    Elegans['Accuracy']['Mackey Glass']['Accuracy Value'].append(accuracy['Elegans'])
    temp=Elegans['Accuracy']['Mackey Glass']['Accuracy Value'][i]
    if temp < 0:
        temp=0
    Elegans_MG_accuracy[i]=temp

In [2]:
#Plot accuracy vs number of cheat steps
plt.plot(Elegans['Accuracy']['Mackey Glass']['Accuracy Value'])
plt.xticks(range(0,len(Elegans['Accuracy']['Mackey Glass']['Cheat Steps']),1),labels=Elegans['Accuracy']['Mackey Glass']['Cheat Steps'])
plt.xlabel('Number of Cheat Steps')
plt.ylabel('Mackey Glass Accuracy')
plt.title('C Elegans | Past Steps = 50 | Cheat Period = 100')

NameError: name 'plt' is not defined

## Watts-Strogatz
### Grid & Random

In [3]:
cheatSteps=range(10,61,10)
mse300_WS_Grid_cheat=[[None]*len(ws300) for i in range(len(cheatSteps))]
numNodes=len(ws300[0][0])
for i in range(len(ws300)): #number of cheat steps
    adjmat=np.asarray(nx.adjacency_matrix(ws300[i][0]).todense())
    electrodes=edamame.core.getFarthestPairing(adjmat)
    for j in range(len(cheatSteps)): #number of networks
        print('Network ' + str(i))
        print('Cheat Steps ' + str(cheatSteps[j]))        
        #run forcast on network j for i cheat steps 
        mse300_WS_Grid_cheat[j][i]=runForecastNew(connectivity__(graph=ws300[i][0]), multi = 1, shift = 0, 
                        training_ratio = 0.3, past_steps = 50,
                        cheat_period = 100, cheat_steps = cheatSteps[j],
                        save_fig = False,
                        disable_tqdm = False,inList=0,outList=int(numNodes/2))

NameError: name 'ws300' is not defined

In [4]:
# mseRandom300=[None]*len(ws300)
cheatSteps=range(10,61,10)
mse300_WS_Random_cheat=[[None]*len(ws300) for i in range(len(cheatSteps))]

for i in range(len(ws300)): #number of cheat steps
    adjmat=np.asarray(nx.adjacency_matrix(ws300[i][1]).todense())
    electrodes=edamame.core.getFarthestPairing(adjmat)
    for j in range(len(cheatSteps)): #number of networks
        print('Network ' + str(i+1))
        print('Cheat Steps ' + str(cheatSteps[j]))        
        #run forcast on network j for i cheat steps 
        mse300_WS_Random_cheat[j][i]=runForecastNew(connectivity__(graph=ws300[i][1]), multi = 1, shift = 0, 
                        training_ratio = 0.3, past_steps = 50,
                        cheat_period = 100, cheat_steps = cheatSteps[j],
                        save_fig = False,
                        disable_tqdm = False, inList=electrodes[0],outList=electrodes[1])

# #     numNodes=ws300[i][1].number_of_nodes()
#     mseRandom300[i]=runForecastNew(connectivity__(graph=ws300[i][0]), multi = 1, shift = 0, 
#                 training_ratio = 0.3, past_steps = 50,
#                 cheat_period = 100, cheat_steps = 60,
#                 save_fig = False,
#                 disable_tqdm = False,inList=electrodes[0],outList=electrodes[1])

   

NameError: name 'ws300' is not defined

In [5]:
accuracy['WS Random 300']=[[None]*len(ws300) for i in range(len(cheatSteps))]
for i in range(len(cheatSteps)):
    for j in range(len(ws300)):
        if i==0:
            WS_Random[j]['Accuracy']['Mackey Glass']={'Cheat Steps':[],'Accuracy Value':[]}
        print(str(cheatSteps[i]))
        WS_Random[j]['Accuracy']['Mackey Glass']['Cheat Steps'].append(cheatSteps[i])
        accuracy['WS Random 300'][i][j]=1-mse300_WS_Random_cheat[i][j]
        WS_Random[j]['Accuracy']['Mackey Glass']['Accuracy Value'].append(1-mse300_WS_Random_cheat[i][j])

accuracy['WS Grid 300']=[[None]*len(ws300) for i in range(len(cheatSteps))]
for i in range(len(cheatSteps)):
    for j in range(len(ws300)):
        if i==0:
            WS_Grid[j]['Accuracy']['Mackey Glass']={'Cheat Steps':[],'Accuracy Value':[]}
        print(str(cheatSteps[i]))
        WS_Grid[j]['Accuracy']['Mackey Glass']['Cheat Steps'].append(cheatSteps[i])
        accuracy['WS Grid 300'][i][j]=1-mse300_WS_Grid_cheat[i][j]
        WS_Grid[j]['Accuracy']['Mackey Glass']['Accuracy Value'].append(1-mse300_WS_Grid_cheat[i][j])

# for i in range(len(cheatSteps)):
#     for j in range(len(mse300_cheat[i])):
#         accuracy['ASN300'][i][j]=1-mse300_cheat[i][j] #save for analysis


#         ASN300[j]['Accuracy']['Mackey Glass']=accuracy['ASN300'][i][j] #save for plotting

NameError: name 'ws300' is not defined

In [ ]:
ASN_MG_accuracy=[[None]*len(ASN300) for i in range(len(cheatSteps))]
for i in range(len(ASN300)):
    temp=[]
    for j in range(len(ASN300[i]['Accuracy']['Mackey Glass']['Accuracy Value'])):
        temp=ASN300[i]['Accuracy']['Mackey Glass']['Accuracy Value'][j]
        if temp < 0:
            temp=0
        if j == 0:
            ASN_MG_accuracy[0][i]= temp
        elif j == 1:
            ASN_MG_accuracy[1][i]= temp
        elif j == 2:
            ASN_MG_accuracy[2][i]= temp
        elif j == 3:
            ASN_MG_accuracy[3][i]= temp
        elif j == 4:
            ASN_MG_accuracy[4][i]= temp
        elif j == 5:
            ASN_MG_accuracy[5][i]= temp

In [6]:
WS_MG_Randomaccuracy=[[None]*len(WS_Random) for i in range(len(cheatSteps))]
for i in range(len(ASN300)):
    temp=[]
    for j in range(len(WS_Random[i]['Accuracy']['Mackey Glass']['Accuracy Value'])):
        temp=WS_Random[i]['Accuracy']['Mackey Glass']['Accuracy Value'][j]
        if temp < 0:
            temp=0
        if j == 0:
            WS_MG_Randomaccuracy[0][i]= temp
        elif j == 1:
            WS_MG_Randomaccuracy[1][i]= temp
        elif j == 2:
            WS_MG_Randomaccuracy[2][i]= temp
        elif j == 3:
            WS_MG_Randomaccuracy[3][i]= temp
        elif j == 4:
            WS_MG_Randomaccuracy[4][i]= temp
        elif j == 5:
            WS_MG_Randomaccuracy[5][i]= temp

NameError: name 'WS_Random' is not defined

In [7]:
WS_MG_Gridaccuracy=[[None]*len(WS_Grid) for i in range(len(cheatSteps))]
for i in range(len(ASN300)):
    temp=[]
    for j in range(len(WS_Grid[i]['Accuracy']['Mackey Glass']['Accuracy Value'])):
        temp=WS_Grid[i]['Accuracy']['Mackey Glass']['Accuracy Value'][j]
        if temp < 0:
            temp=0
        if j == 0:
            WS_MG_Gridaccuracy[0][i]= temp
        elif j == 1:
            WS_MG_Gridaccuracy[1][i]= temp
        elif j == 2:
            WS_MG_Gridaccuracy[2][i]= temp
        elif j == 3:
            WS_MG_Gridaccuracy[3][i]= temp
        elif j == 4:
            WS_MG_Gridaccuracy[4][i]= temp
        elif j == 5:
            WS_MG_Gridaccuracy[5][i]= temp

NameError: name 'WS_Grid' is not defined

In [8]:
labels_MG=['10 Cheat Steps','20 Cheat Steps','30 Cheat Steps','40 Cheat Steps','50 Cheat Steps','60 Cheat Steps']

In [ ]:
#MG Plot SmallWorldness:
fig1=plt.figure(figsize=(15,15))
ax=[None]*len(cheatSteps)
axBig = fig1.add_subplot(111)
axBig.set_frame_on(False)
axBig.set_yticklabels([])
axBig.set_xticklabels([])
axBig.set_xticks([])
axBig.set_yticks([])
plt.xlabel('Small World Propensity',fontsize=30,labelpad=15)
plt.ylabel('Accuracy',fontsize=30,labelpad=15)
plt.title('Mackey Glass Performance',fontsize=30,pad=30)
for i in range(len(ax)):
    ax[i]=fig1.add_subplot(2, 3, i+1)
    plt.scatter(smallworld,ASN_MG_accuracy[i],label='ASN',marker='s')
    plt.scatter(smallworld_random,WS_MG_Randomaccuracy[i],label='WS Random',marker='o')
    plt.scatter(smallworld_grid,WS_MG_Gridaccuracy[i],label='WS Grid',marker='o')
    plt.scatter(Elegans['Graph Theory']['Small World'],Elegans_MG_accuracy[i],label='Elegans',marker='x')#,0,24,label='Elegans')
    plt.title(str(labels_MG[i]))
    plt.legend(loc='bottom right')
    plt.ylim(0,1)

#DOUBLE CHECK SMALL WORLDNESS 
    
#      #Add correlation line
      #  ASN300
    x=np.array(ASNsw, dtype=np.float)
    y=np.array(ASN_MG_accuracy[i], dtype=np.float)
    idx = np.isfinite(x) & np.isfinite(y)
    m, b = np.polyfit(x[idx],y[idx], 1)
    X_plot = np.linspace(ax[i].get_xlim()[0],ax[i].get_xlim()[1],100)
    plt.plot(X_plot, m*X_plot + b, '-')
#      #WS Random
#     x=np.array(WS_Randsw, dtype=np.float)
#     y=np.array(WS_MG_Randomaccuracy[i], dtype=np.float)
#     idx = np.isfinite(x) & np.isfinite(y)
#     m, b = np.polyfit(x[idx],y[idx], 1)
#     X_plot = np.linspace(ax[i].get_xlim()[0],ax[i].get_xlim()[1],100)
#     plt.plot(X_plot, m*X_plot + b, '-')
    
#     x=np.array(WS_Gridsw, dtype=np.float)
#     y=np.array(WS_MG_Gridaccuracy[i], dtype=np.float)
#     idx = np.isfinite(x) & np.isfinite(y)
#     m, b = np.polyfit(x[idx],y[idx], 1)
#     X_plot = np.linspace(ax[i].get_xlim()[0],ax[i].get_xlim()[1],100)
#     plt.plot(X_plot, m*X_plot + b, '-')
    plt.savefig(r'C:\Users\aloe8475\Documents\PhD\GitHub\CODE\Data\Figures\Functional Connectivity\'Mackey Glass Accuracy vs Small World Prop.jpg')

#     plt.savefig()